# First Cell 

## makes sure the yfinance is the latest version and that yfinance, numpy and pandas are imported

In [ ]:
%pip install --upgrade yfinance

import yfinance as yf
import pandas as pd
import numpy as np

Note: you may need to restart the kernel to use updated packages.


### This cell defines the tickers and downloads them from Yfinance.

It also prints the first 5 lines of the data to prove that it was downloaded

In [29]:


# Define the tickers, including the S&P 500
tickers = [ "AAPL","AMZN","GOOG","NKE","RY.TO","^GSPC"]  # RY.TO is for Royal Bank of Canada, ^GSPC is the S&P 500

# Download historical data (e.g., for the past year)
data = yf.download(tickers, period="2y")

print(data.head(5))



[*********************100%***********************]  6 of 6 completed

Price            Close                                                \
Ticker            AAPL       AMZN       GOOG         NKE       RY.TO   
Date                                                                   
2023-02-21  146.997559  94.580002  91.720444  117.538406  127.733490   
2023-02-22  147.423294  95.790001  91.471336  116.373718  126.181068   
2023-02-23  147.908371  95.820000  90.743942  116.431946  125.552696   
2023-02-24  145.245255  93.500000  89.030098  114.568420  126.800201   
2023-02-27  146.443146  93.760002  89.777420  115.044014  127.557892   

Price                          High                                    ...  \
Ticker            ^GSPC        AAPL       AMZN       GOOG         NKE  ...   
Date                                                                   ...   
2023-02-21  3997.340088  149.789411  95.610001  93.080555  120.353118  ...   
2023-02-22  3991.050049  148.452904  97.010002  92.029329  117.829603  ...   
2023-02-23  4012.320068  148.8389

## This cell calculates the daily returns based on the Closing price and also develops shows a correlation matrix

In [30]:
# --- Calculate Daily Returns ---
returns = data['Close'].pct_change()
print(returns.head(15))
# --- Calculate Correlations ---
correlation_matrix = returns.corr()
print(correlation_matrix)

Ticker          AAPL      AMZN      GOOG       NKE     RY.TO     ^GSPC
Date                                                                  
2023-02-21       NaN       NaN       NaN       NaN       NaN       NaN
2023-02-22  0.002896  0.012793 -0.002716 -0.009909 -0.012154 -0.001574
2023-02-23  0.003290  0.000313 -0.007952  0.000500 -0.004980  0.005329
2023-02-24 -0.018005 -0.024212 -0.018887 -0.016005  0.009936 -0.010538
2023-02-27  0.008247  0.002781  0.008394  0.004151  0.005975  0.003073
2023-02-28 -0.003448  0.005013  0.002220  0.002194  0.003622 -0.003036
2023-03-01 -0.014246 -0.021861  0.002326 -0.001768 -0.035513 -0.004725
2023-03-02  0.004129 -0.000434  0.019887  0.008433  0.019383  0.007582
2023-03-03  0.035090  0.030066  0.018524  0.014257  0.003964  0.016148
2023-03-06  0.018539 -0.012118  0.016592 -0.006367  0.003364  0.000687
2023-03-07 -0.014496 -0.002133 -0.014752 -0.004827 -0.004883 -0.015327
2023-03-08  0.008377  0.003955  0.005097  0.002258  0.003808  0.001415
2023-0

/var/folders/v7/k3z3dzlx5rv8j1g_txkst4vm0000gn/T/ipykernel_18171/3999015160.py:2: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = data['Close'].pct_change()


## Calculate Volatility, Returns and Sharpe Ratio

This also is where the risk free rate is downloaded. It uses the ten year treasury bond rate


In [31]:


# --- Calculate Rolling Volatility (e.g., 30-day) ---
volatility = returns.rolling(window=30).std()

# --- Calculate Cumulative Returns ---
cumulative_returns = (1 + returns).cumprod() - 1

# --- Risk-Free Rate (Example: 10-year US Treasury yield - you'll want to update this) ---
risk_free_rate = yf.download("^TNX", period="1y")['Close'].iloc[-1] / 100 # Annualized risk-free rate.  Divide by 100 to convert percentage to decimal.
# Important: ^TNX is for the 10-year US Treasury.  You might want to use a more appropriate risk-free rate for your analysis (e.g., a Canadian government bond yield if analyzing Canadian stocks).

# --- Sharpe Ratio Calculation (Annualized) ---
def calculate_sharpe_ratio(returns, risk_free_rate, annualization_factor=252): # 252 trading days in a year
    excess_returns = returns - risk_free_rate/annualization_factor # Daily excess return
    annualized_excess_returns = excess_returns.mean() * annualization_factor # Annualize excess return
    annualized_volatility = volatility.mean() * np.sqrt(annualization_factor) # Annualize volatility
    sharpe_ratio = annualized_excess_returns / annualized_volatility
    return sharpe_ratio


sharpe_ratios = calculate_sharpe_ratio(returns, risk_free_rate)

#print("this is the volatiltiy")
#print(volatility.head(50-60))
#print( "this is the cum returns")
#print(cumulative_returns.head(50-60))
#print("this is the risk free rate")
#print( risk_free_rate.head(5))
print("this is the Sharpe ratio")
print(sharpe_ratios)


[*********************100%***********************]  1 of 1 completed

this is the Sharpe ratio
Ticker
AAPL    NaN
AMZN    NaN
GOOG    NaN
NKE     NaN
RY.TO   NaN
^GSPC   NaN
^TNX    NaN
dtype: float64


##This is the export to excel section

In [32]:

# --- Export to Excel (separate sheets) ---
with pd.ExcelWriter("stock_analysis.xlsx") as writer:
    data['Close'].to_excel(writer, sheet_name="Close Prices")  # Adjusted Close Prices
    returns.to_excel(writer, sheet_name="Daily Returns")
    correlation_matrix.to_excel(writer, sheet_name="Correlation Matrix")
    volatility.to_excel(writer, sheet_name="Volatility")
    cumulative_returns.to_excel(writer, sheet_name="Cumulative Returns")
    pd.DataFrame(sharpe_ratios, columns=['Sharpe Ratio']).to_excel(writer, sheet_name="Sharpe Ratios") # Sharpe Ratios


print("Data and analysis saved to stock_analysis.xlsx")

# --- Example of Accessing Data (for us) ---
#print("\nCorrelation Matrix:")
#print(correlation_matrix)

#print("\nVolatility:")
#print(volatility)

#print("\nCumulative Returns:")
#print(cumulative_returns)

# --- Further Analysis (Examples for us) ---
# ... (as before)

# Example: Analyzing correlations with the S&P 500
sp500_corr = correlation_matrix["^GSPC"]
print("\nCorrelation with S&P 500:")
print(sp500_corr)

print("\nSharpe Ratios:")
print(sharpe_ratios)

Data and analysis saved to stock_analysis.xlsx

Correlation with S&P 500:
Ticker
AAPL     0.578493
AMZN     0.632752
GOOG     0.562419
NKE      0.307509
RY.TO    0.450092
^GSPC    1.000000
Name: ^GSPC, dtype: float64

Sharpe Ratios:
Ticker
AAPL    NaN
AMZN    NaN
GOOG    NaN
NKE     NaN
RY.TO   NaN
^GSPC   NaN
^TNX    NaN
dtype: float64
